In [6]:
# Chrome의 경우 | 아까 받은 chromedriver의 위치를 지정해준다.
import sys
import os
if not sys.modules.get('selenium'):
    os.system('pip3 install selenium')
from selenium import webdriver
if not sys.modules.get('requests'):
    os.system('pip3 install requests')
    import requests
if not sys.modules.get('bs4'):
    os.system('pip3 install bs4')
    from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
from selenium.webdriver.common.action_chains import ActionChains
import numpy as np
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
# expected_conditions는 Selenium 2.26.0 이후 부터 사용 가능합니다.
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import re 
if not sys.modules.get('pyautogui'):
    os.system('pip3 install pyautogui')
    import pyautogui
import openpyxl
from tqdm import tqdm

In [7]:
#네이버 백과사전 사이트 접속이 3초 소요된다.
driver = webdriver.Chrome("./chromedriver.exe")
driver.implicitly_wait(3)
driver.get('https://terms.naver.com/list.nhn?cid=60407&categoryId=60407&page=1')

In [8]:
# 총 3584개의 병명 url을 가진 엑셀파일을 불러온다. 엑셀파일명은 disease_url2이다.
urls = pd.read_excel("./naver_excel_file/disease_url2.xlsx")
urls = urls.iloc[:,[2,3]]
urls

,title,url
0,우울증 [depressive disorder],https://terms.naver.com/entry.nhn?docId=211985...
1,양성자세현훈 [benign paroxysmal positional vertigo],https://terms.naver.com/entry.nhn?docId=926692...
2,어지럼증 [dizziness],https://terms.naver.com/entry.nhn?docId=211961...
3,협심증 [angina pectoris],https://terms.naver.com/entry.nhn?docId=211994...
4,자궁근종 [myoma uteri],https://terms.naver.com/entry.nhn?docId=211992...
...,...,...
3579,임신성 당뇨병의 개념,https://terms.naver.com/entry.nhn?docId=556866...
3580,미숙아망막병증,https://terms.naver.com/entry.nhn?docId=556866...
3581,GM₃-강글리오시드증 [GM₃-gangliosidosis],https://terms.naver.com/entry.nhn?docId=564750...
3582,염증성 장질환을 진단하는 검사 중 산모에게 안전한 것은 무엇입니까?,https://terms.naver.com/entry.nhn?docId=283642...


In [82]:
# 본문내용의 html태그를 제거하는 정규표헌식
import re
regex = re.compile("<(.*?)>")

In [123]:
# 크롤링한 내용을 data_frame에 저장하기 위해서 비어있는 데이터프레임을 만듭니다.
data_frame = pd.DataFrame()
# 엑셀파일의 url들을 차례대로 불러서 크롤링을하고 50개마다 엑셀파일로 저장하겠습니다.
for j in tqdm(range(2400,len(urls))):
    # 불러온 url로 이동합니다.
    driver.get(urls.url[j])
    # 2초 기다립니다.
    time.sleep(2)
    # 동영상이 있으면 다음 url로 넘아갑니다.
    try:
        driver.find_element_by_css_selector("#size_ct > div > div.na_video")
        driver.find_element_by_css_selector("#size_ct > div.video_thmb")
    except:
        driver.get(urls.url[j+1])
        time.sleep(2)
    # 900개쯤 지나가니까 사이트를 찾지못하는 오류가 발생하여, 새로고침 후 2초 기다리고 다시합니다.
    try:
        contents = driver.find_element_by_css_selector("#content > div.section_wrap")
    except:
        driver.refresh()
        time.sleep(3)
        contents = driver.find_element_by_css_selector("#content > div.section_wrap")
    # 병명을 저장합니다.
    disease_name = contents.find_element_by_css_selector("h2").text
    # selenium으로 가져오지 못한 경우 beatifulsoup로 전체 page_source를 가져와서 html태그를 제거합니다.
    try:
        # html태그가 p에 해당하는 부분만 가져옵니다.
        full_contents = contents.find_elements_by_css_selector("p")
        # 가져온 형태가 여러개의 리스트형태이기에 하나의 리스트로 다시 만듭니다.
        full_contents = [x.text for x in full_contents]
        # 하나의 리스트를 하나의 문자열로 합칩니다.
        full_contents = " ".join(full_contents)
    except:
        # 전체 html태그를 가져옵니다.
        html3 = driver.page_source
        # 파싱합니다.
        soup3 = BeautifulSoup(html, 'html.parser')
        # 파싱후 본문내용에 해당하는 html태그를 선택하여 가져옵니다.
        full_contents = soup3.select("#size_ct")
        # 해당 본문 내용에는 본내용과 html태그등이 섞여있어서 필요없는건 모두 제거합니다.
        full_contents = regex.sub("",full_contents)
        full_contents = re.sub("\n"," ",full_contents)
    try:
        # 해당내용의 출처를 혹시 몰라 가져옵니다.
        source = contents.find_element_by_css_selector("div.section_area > div").text
    except:
        pass
    # 목차내용에 맞게 내용을 분류합니다. 가끔 목차가 없는 경우가 있어 에러가 나면 바로 넘어갑니다.
    try:
        # 목차에 해당하는 html태그를 가져옵니다.
        preview = driver.find_element_by_css_selector("#size_ct > div.tmp_agenda > ol")
        detail_contents = preview.find_elements_by_css_selector("li")
        detail_contents = [y.text for y in detail_contents]
        # 목차 갯수를 기준으로 목차에 맞는 세부내용을 알맞은 dictionary형태로 저장합니다.
        json = {}
        for i in range(0,len(detail_contents)):
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser') 
            te = soup.select('#size_ct')
            ste = str(te)
            # 세부내용이 page_source로 보니 <h3 class="stress"의 시작점과 끝점으로 나뉩니다.
            index = [(m.start(0), m.end(0)) for m in re.finditer('<h3 class="stress"',ste)]
            # 가져온 세부내용을 보기좋게 전처리하고 비어있는 dictionary에 update 시킵니다.
            try:
                # 목차가 4개인 경우 1~3까지 가져와서 update합니다.
                detail_con = regex.sub("",ste[index[i][0]:index[i+1][0]])
                detail_con = re.sub("\n"," ",detail_con)
                json.update({detail_contents[i] : detail_con})
            except:
                # 3~마지막까지 가져와서 update 합니다.
                detail_con1 = regex.sub("",ste[index[i][0]:len(ste)])
                detail_con1 = re.sub("\n"," ",detail_con1)
                json.update({detail_contents[i] : detail_con1})
        # 위에서 가져온 병명,내용,세부내용등을 모두 데이터프레임으로 만들고 합칩니다.
        detail = pd.DataFrame(json,index=[0])
        con = pd.DataFrame({"name":disease_name,"contents":full_contents},index=[0])
        con_detail = pd.concat((con,detail),axis=1)
        # 맨 처음에는 비어있는 데이터프레임이기에 merge가 안되어 맨처음 데이터프레임을 채워넣습니다.
        if j == 2400:
            data_frame = con_detail
        # 이전 데이터프레임과 현재 데이터프레임을 합치는데 열이 안맞는 곳은 na로 저장이 됩니다.
        data_frame = pd.merge(data_frame,con_detail,how="outer")
    except:
        con = pd.DataFrame({"name":disease_name,"contents":full_contents},index=[0])
        data_frame = pd.merge(data_frame,con,how="outer")
    # 50번쨰 url마다 체크포인트 마냥 엑셀파일을 저장합니다.
    try:
        if j % 100 == 0:
            data_frame.to_excel("./data/naver_disease"+str(j)+".xlsx")
    except:
        pass
# 모두 완료가 되면 최종 데이터프레임을 엑셀파일로 저장합니다.    
data_frame.to_excel("./data/naver_disease"+str(len(urls))+".xlsx")

 22%|████████████████▊                                                            | 323/1483 [48:06<2:52:46,  8.94s/it]


TypeError: expected string or bytes-like object

In [5]:
import pandas as pd
aa = pd.read_excel("C:/Users/park/Desktop/repo/data_campus/naver_disease_dict/naver_excel_file/naver_disease2400.xlsx",index=False)
del aa["Unnamed: 0"]
aa

,name,contents,개요,증상,원인,진단,치료,참고문헌 및 사이트,예방,대상별 맞춤정보,...,환자 및 접촉자 관리,예방/조기검진,부위별 염좌,위험요인,합병증,적정체중 유지와 체중 감량,관절에 좋은 적절한 활동,관절의 부상에 주의,관절을 보호하는 생활습관,약물이나 식이요법
0,다초점 운동 신경병증,희귀질환정보 [ Multifocal motor neuropathy ] 신경전도차단...,개요 신경전도차단을 동반한 다초점 운동 신경병증(MMN)은 후천적으로 서서히 진행하...,"증상 일반적으로, 다초점 운동 신경병증(MMN)은 비대칭적으로 서서히 진행하며, 몇...",원인 다초점 운동 신경병증(MMN)은 발병 원인이 밝혀지지 않은 자가면역성의 말초신...,진단 1. 진단검사의학과 검사항-갱글리오사이드(Anti-GM1) 항체크레아틴키나아제...,치료 1. 의학적 치료 다초점 운동 신경병증(MMN)는 서서히 진행되는 근육 약화 ...,"참고문헌 및 사이트 Yuki N, Watanabe H, Nakajima T, Spa...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,반코마이신내성장알균(VRE) 감염증,"국가건강정보포털 의학정보 신고범위 : 환자, 병원체보유자 신고를 위한 진단기준 ...",NaN,증상 장알균은 위장관과 비뇨생식계에 존재하는 상재균으로 정상인에서는 쉽게 발병...,NaN,진단 아래의 경우 반코마이신 내성 장알균 감염증을 진단할 수 있습니다. • 혈액에서...,치료 일반적으로 감염 증상이 나타나지 않은 환자에게 치료는 필요하지 않습니다. 감염...,NaN,예방 반코마이신 내성 장알균의 감염 또는 보균자에 대하여 환자 격리를 실시하고 전파...,"대상별 맞춤정보 환자 및 접촉자 관리환자와 접촉하는 사람의 경우 처치 전후 손씻기,...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,스미스-렘리-오피츠 증후군,희귀질환정보 [ Smith-Lemli-Opitz syndrome ] 스미스-렘리-...,"개요 스미스-렘리-오피츠 증후군(SLOS)은 1964년에 스미스, 렘리, 오피츠에 ...",증상 스미스-렘리-오피츠 증후군의 증상은 개개인의 경우에 따라 매우 다르게 나타납니...,원인 스미스-램리-오피츠 증후군은 콜레스테롤 대사에 관여하는 7-dehydrocho...,진단 1. 혈액검사 7-dehydrocholesterol(7-DHC)의 농도가 증가...,치료 스미스-렘리-오피츠 증후군의 치료는 환자들에게 나타나는 증상에 맞추어 치료를 ...,"참고문헌 및 사이트 Goodwin H, Brooks BP, Porter FD. Ac...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,노인 약물복용,국가건강정보포털 의학정보 노화현상이 질병은 아니지만 나이가 들수록 질병에 걸릴 ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,쯔쯔가무시병,국가건강정보포털 의학정보 [ scrub typhus\n음성듣기\n] 신고범위 : ...,개요 쯔쯔가무시균(Orientia tsutsugamushi) 감염에 의한 급성 발...,NaN,원인 쯔쯔가무시병의 감염 경로 쯔쯔가무시병의 매개체인 털진드기는 알→유충→약충→성충...,"진단 일반적으로 쯔쯔가무시병은 대부분 가을철에 발생하며, 야외 활동력과 함께 가피와...",치료 1. 약물요법 쯔쯔가무시병의 치료는 독시싸이클린(doxycycline)이 선택...,NaN,예방 1. 일반적 예방 방법 진드기에 물리지 않는 것이 쯔쯔가무시병을 예방하는 최선...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
286,조기치료의 기적! 조기정신증,정신이 건강해야 삶이 행복합니다 조현병은 발병 이전에 명백하진 않지만 정신이상 ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
287,건강검진에서 LDL 콜레스테롤가 정상수치(120mg/dL)로 나와 약을 복용하지 않...,혈관건강백서 약물치료는 심혈관질환 위험도와 LDL 콜레스테롤의 수치를 종합적으로...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
288,기관지 폐포암,서울대학교병원 의학정보 [ bronchioloalveolar carcinoma ]...,NaN,증상 기관지 폐포암은 다른 폐암에 비하여 진행 속도가 훨씬 느리다. 따라서 다른 폐...,원인 기관지 폐포암도 다른 폐암처럼 흡연과 연관이 있지만 다른 폐암에 비하여 그 영...,NaN,"치료 초기의 환자는 수술로 치료하며, 방사선 치료, 항암 화학요법에 대한 반응은 다...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
289,몸을 돌보면 마음도 건강해진다.,"정신이 건강해야 삶이 행복합니다 건강한 몸은 정신의 전당이고, 병든 몸은 정신의...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [124]:
json

{'정의': '정의 선천성 근육병증(congenital myopathy)은 유전적인 요인에 의해 발생하는 근육병증으로, 대부분 진행성 근력저하가 나타나는 질환이다. 선천성 근육병증을 유발하는 유전자는 매우 다양하므로 각 질환마다 침범하는 근육의 위치, 근육 쇠약의 정도, 근육병증 발현의 시기, 병의 경과 및 치료 결과 등이 서로 다르다. ',
 '원인': '원인 선천성 근육병증은 유전적인 요인에 의하여 발생한다. 각 질환에 따라 원인이 되는 유전자는 서로 다르며, 동일한 증상을 나타내는 질환이라도 서로 다른 유전자가 관여하는 경우도 있다. 이상이 생긴 유전자에 따라 유전 형태도 서로 다르다.선천성 근육병증 이외에 근육디스트로피(muscular dystrophy) 또한 유전적인 원인에 의해 발생하는 근육병증의 한 종류이다. 근육디스트로피는 주로 디스트로핀(dystrophin) 단백질 및 이와 관련된 근세포막의 디스트로핀-당단백 복합체(dystrophin-glycoprotein complex)의 이상으로 발생한다. 디스트로핀-당단백 복합체는 근육 내의 골격을 형성하는 액틴(actin)이라는 단백질을 근세포막에 부착시키는 역할을 하며, 이 복합체의 기능에 이상이 발생할 경우 근육세포의 퇴행 및 괴사가 발생한다.과거 근육디스트로피는 근막에 존재하는 단백질의 소실에 의해서만 발생한다고 생각되었지만, 최근 핵에 존재하는 단백질 및 근세포 내의 효소 등의 이상에 의해서도 발생할 수 있음이 알려지고 있다.  ',
 '증상': '증상 선천성근육병증은 드물며 매우 이질적인 질환군이다. 주로 유아 시절에 시작되고 가족력이 있으며 서서히 진행하는 특성을 지니지만, 성인에게서 발병하기도 하고 가족력이 없는 사람에게 나타나기도 하며 매우 빠른 진행 양상을 보이기도 한다. 주로 발달장애나 팔다리의 근력저하가 첫 증상으로 나타나며, 심한 경우에는 영아 시기에 호흡근의 근력 저하로 인한 호흡곤란으로 사망하기도 한다.중심핵병(central core disease)은 임상적으로 근육 긴장 저하